<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex10/Antonio_Angulo/Aula_10_Exerc%C3%ADcio_Antonio_Angulo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome ='Antonio L. Angulo Salas'
print(f'Meu nome é {nome}')

Meu nome é Antonio L. Angulo Salas


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 14.2 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.1 MB/s 
     |████████████████████████████████| 596 kB 72.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Thu Jun  9 00:28:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


def truncate_and_pad(batch_word_ids, pad_token_id, seq_length=9):  

    lens = [len(l) for l in batch_word_ids] 
    arr_zeros = np.zeros((len(batch_word_ids),seq_length),int) 
    
    mask = np.arange(seq_length) < np.array(lens)[:,None]

    batch_word = arr_zeros + pad_token_id
    batch_word[mask] = np.concatenate([i if len(i) < seq_length else i[0:seq_length]  for i in batch_word_ids] ) 
    mask =  1*mask

    return batch_word, mask

class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        # Escreva aqui seu código.
        self.examples = []
        for text in texts:
          token_ids = tokenize(text=text, tokenizer=tokenizer)
          
          for j in range(len(token_ids)//(max_seq_length-1)+1):
            input_ids = tokenize(text='[CLS]', tokenizer=tokenizer) + token_ids[j*(max_seq_length-1):(j+1)*(max_seq_length-1)]
            target_id = token_ids[j*(max_seq_length-1):(j+1)*(max_seq_length-1)+1]

            if (len(input_ids) < max_seq_length):
              input_ids = input_ids + [tokenizer.pad_token_id]*(max_seq_length-len(input_ids))

            if (len(target_id) < max_seq_length):
              target_id = target_id + [tokenizer.pad_token_id]*(max_seq_length-len(target_id))
            
            self.examples.append((input_ids, target_id))
           

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.tensor(input_ids), torch.tensor(target_id)

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])

assert torch.equal(first_batch_input, correct_first_batch_input)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

Passou no assert de tamanho do dataset.
Passou no assert de dataset.


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-09 00:28:22--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.18.128, 74.125.128.128, 173.194.69.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.18.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G   142MB/s    in 6.8s    

2022-06-09 00:28:29 (174 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [9]:
# Load datasets
max_seq_length = 9

train_examples = 5000
valid_examples = 100
test_examples = 100

texts = open('sample-1gb.txt').readlines()

print(f'Read {len(texts)} lines.')

max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')
texts = texts[:max_lines] 


training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)

Read 250000 lines.
Truncating to 5200 lines.


In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 736640
valid examples: 10481
test examples: 15756


In [11]:
import gensim

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
        """
        # Escreva seu código aqui.
        super().__init__()

        self.max_seq_length = max_seq_length
        self.embeddings_dim = dim        
        self.vocab_size = vocab_size
        self.pad_token_id = pad_token_id
        self.embeddings = nn.Embedding(vocab_size, dim)
        self.n_layers = n_layers

        self.W_Q = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False) 
        self.W_K = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False) 
        self.W_V = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False)

        self.W_O = nn.Linear(self.embeddings_dim, self.embeddings_dim, bias=False)

        hidden_size = 1024

        self.layer1 = nn.Linear(self.embeddings_dim, hidden_size)
        self.layer2 = nn.Linear(hidden_size,vocab_size)

    def Attention(self, q, k, v, inputs):

        scores = torch.matmul(q, k.transpose(2,1))
        m_pad = inputs != self.pad_token_id
        scores = scores.masked_fill(~m_pad[:, None], -1e9)

        m_causal = torch.tril(torch.ones(self.max_seq_length, self.max_seq_length)).to(device)
        m_causal = m_causal.bool()
        scores = scores.masked_fill(~m_causal, -1e9)

        probs =  F.softmax(scores, dim=-1) 
        Atenttion_Head  = torch.matmul(probs, v) 

        return Atenttion_Head

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        # Escreva seu código aqui.

        X = self.embeddings(inputs)

        for f in range(0, self.n_layers):
          residual = X

          q = self.W_Q(X).to(device) 
          k = self.W_K(X).to(device) 
          v = self.W_V(X).to(device) 

          X = self.Attention(q=q, k=k, v=v, inputs = inputs)

          X = self.W_O(X).to(device)

          X += residual

        hidden = torch.relu(self.layer1(X))
        
        logits = self.layer2(hidden)

        return logits

## Teste o modelo com um exemplo

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [13]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 32528610


## Assert da Perplexidade


In [14]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    print(logits.shape)
    print(target.shape)
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model(train_input_ids)

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

torch.Size([9000, 29794])
torch.Size([9000])
my perplexity:              31187
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [15]:
max_examples = 50_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    pad_token_id=tokenizer.pad_token_id,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=1024)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=tokenizer.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=tokenizer.pad_token_id)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for train_input_ids, train_target_ids in train_loader:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30920.19, valid ppl: 28119.26
10000 steps; 10240000 examples so far; train ppl: 341.44, valid ppl: 396.89


KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [16]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 434.53027455240954


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [17]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz parte
Eu gosto de comer pizza pois me faz parteha
Eu gosto de comer pizza pois me faz parteha,
Eu gosto de comer pizza pois me faz parteha, e
Eu gosto de comer pizza pois me faz parteha, e de
Eu gosto de comer pizza pois me faz parteha, e de um
Eu gosto de comer pizza pois me faz parteha, e de um dos
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e de
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e de uma
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e de uma das
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e de uma das famílias
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e de uma das famílias,
Eu gosto de comer pizza pois me faz parteha, e de um dos maiores e de uma das famílias, e
Eu gosto de comer pizza pois me faz parteha

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.